## halo_BCG_matching
#### Created Sept. 20, 2019

In [19]:
import numpy as np
import sys
sys.path.append('/home/zzbenjamin94/Desktop/Astronomy/Research/DES_Galaxy_Cluster')
from tools.setup.setup import *
home_dir = home_dir()
datadir = data_home_dir()
bigdatadir = bigdata_home_dir()

# import pyplot and set some parameters to make plots prettier
import matplotlib.pyplot as plt
from tools.plot_utils import plot_pretty
plot_pretty()

# setup notebook for inline figures
%matplotlib inline

from tools.setup.setup import *
import astropy.io.fits as fits
#Read http://pythonhosted.org/pyfits/

## Import Files
### Import Halo File

In [18]:
#Only record array pickle file
import pickle
halos_file = bigdatadir + 'Buzzard_Sims/v1.6/buzzard_halo/halos/' + 'Chinchilla-0_halos_unique.all.fits'
halos_list = fits.open(halos_file)
halos_data = halos_list[1].data

#Checking FITS into
#hrun_list.info()
print "Names of columns in halos_data are:"
print halos_data.dtype

#Converting FITS files into record array
halos_data = np.array(halos_data, dtype=halos_data.dtype)
halos_data = halos_data.view(np.recarray)

print "\n Number of halos is", len(halos_data.ID)

Names of columns in halos_data are:
(numpy.record, [('ID', '>i8'), ('DESCID', '>i8'), ('MVIR', '>f8'), ('VMAX', '>f8'), ('VRMS', '>f8'), ('RVIR', '>f8'), ('RS', '>f8'), ('NP', '>f8'), ('PX', '>f8'), ('PY', '>f8'), ('PZ', '>f8'), ('VX', '>f8'), ('VY', '>f8'), ('VZ', '>f8'), ('JX', '>f8'), ('JY', '>f8'), ('JZ', '>f8'), ('SPIN', '>f8'), ('RS_KLYPIN', '>f8'), ('MVIR_ALL', '>f8'), ('M200B', '>f8'), ('M200C', '>f8'), ('M500C', '>f8'), ('M2500C', '>f8'), ('XOFF', '>f8'), ('VOFF', '>f8'), ('SPIN_BULLOCK', '>f8'), ('B_TO_A', '>f8'), ('C_TO_A', '>f8'), ('PID', '>i8'), ('LUMTOT', '>f8'), ('LUM20', '>f8'), ('LBCG', '>f8'), ('NGALS', '>i8'), ('N18', '>i8'), ('N19', '>i8'), ('N20', '>i8'), ('N21', '>i8'), ('N22', '>i8'), ('Z', '>f8'), ('RA', '>f8'), ('DEC', '>f8')])

 Number of halos is 12198813


### Import BCG File

In [20]:
BCG_filename = datadir + 'v1.6/BCG_v1.6_info.data' 
BCG_file = open(BCG_filename, "rb" )
BCG_data = pickle.load(BCG_file)
BCG_data = BCG_data.view(np.recarray)

In [21]:
##Testing BCG file properties. 
print BCG_data.dtype
print "Number of BCGs are ", len(BCG_data)

(numpy.record, [('ID', '<i8'), ('HALOID', '<i8'), ('RA', '<f4'), ('TRA', '<f4'), ('DEC', '<f4'), ('TDEC', '<f4'), ('Z', '<f4')])
Number of BCGs are  11161403


## Create BCG Fits File from Record Array

In [32]:
from astropy.io import fits
##Table datatypes
#K: 64 bit integer
#E: single precision float
#D: double precision float

c1 = fits.Column(name='ID', array=BCG_data.ID, format='K')
c2 = fits.Column(name='HALOID', array=BCG_data.HALOID, format='K')
c3 = fits.Column(name='RA', array=BCG_data.RA, format='D')
c4 = fits.Column(name='TRA', array=BCG_data.TRA, format='D')
c5 = fits.Column(name='DEC', array=BCG_data.DEC, format='D')
c6 = fits.Column(name='TDEC', array=BCG_data.TDEC, format='D')
c7 = fits.Column(name='Z', array=BCG_data.Z, format='D')

t = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7])
t.writeto('BCG_v1.6_data.fits')

## Match halos to BCGs by RA, DEC

In [31]:
# Finding mismatch between RA, DEC, Z in halorun versus BCG_galaxy
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import WMAP9 as cosmo
from astropy.coordinates import SkyCoord

#Using find_nearest iterate through the array and find index of matched ones, also index of unmatched. 
#Go through the richness-mass template finder. 
BCG_coord = SkyCoord(BCG_data.TRA*u.deg, BCG_data.TDEC*u.deg,  frame='icrs')
halos_coord = SkyCoord(halos_data.RA*u.deg, halos_data.DEC*u.deg, frame='icrs')

#Finding nearest neighbor to the halorun catalog.
def find_nearest(ind):
    dist_sep = halos_coord.separation(BCG_coord[ind])
    idx = (np.abs(dist_sep)).argmin()
    return idx, dist_sep[idx].value

#Testing match of specific halo using index testn
testarr = np.arange(0,1000,100)
for i, testn in enumerate(testarr):
    ind, dist = find_nearest(testn)
    print "BCG index & ID: ", testn, BCG_data.HALOID[ind]
    print "Halo matched index & ID:", ind, halos_data.ID[testn]
    print "Total angular separation: ", dist
    print "RA difference: ", halos_data.RA[ind] -  BCG_data.TRA[testn]
    print "DEC difference: ", halos_data.DEC[ind] - BCG_data.TDEC[testn]
    #print hrun_data.Z[testn] - redM_gal_data.Z[ind] #Some error in redshift
    print "Redshift difference: {:2g} \n".format(BCG_data.Z[testn] - halos_data.Z[ind]) #Small error in redshift


BCG index & ID:  0 1008492304
Halo matched index & ID: 5624780 4528806
Total angular separation:  4.39557663776e-06
RA difference:  -4.424237744160564e-06
DEC difference:  -1.9649285576406328e-07
Redshift difference: 9.49143e-06 

BCG index & ID:  100 1010531327
Halo matched index & ID: 5625069 3415599
Total angular separation:  4.40421478078e-06
RA difference:  4.4230769447040075e-06
DEC difference:  -7.944066382492565e-08
Redshift difference: 7.93817e-06 

BCG index & ID:  200 1008419345
Halo matched index & ID: 5624968 3449708
Total angular separation:  8.12207190894e-07
RA difference:  -4.4125837916908495e-07
DEC difference:  -6.842911846050015e-07
Redshift difference: 9.08883e-06 

BCG index & ID:  300 130075054
Halo matched index & ID: 5622663 5053423
Total angular separation:  6.35748174879e-06
RA difference:  -6.372460433112792e-06
DEC difference:  1.1903274277358378e-07
Redshift difference: 8.12658e-06 

BCG index & ID:  400 1008515446
Halo matched index & ID: 5626197 5053183


KeyboardInterrupt: 

In [30]:
#Finding BCG's nearest halo neighbor.
def find_nearest(ind):
    dist_sep = halos_coord.separation(BCG_coord[ind])
    idx = (np.abs(dist_sep)).argmin()
    return idx, dist_sep[idx].value

BCG_matched_ID = [] 
redM_matched_ID = []

#Find the centered ones
ang_lim = 1e-5
z_lim = 0.01
BCG_coord = SkyCoord(BCG_data.TRA*u.deg, BCG_data.TDEC*u.deg,  frame='icrs')
halos_coord = SkyCoord(halos_data.RA*u.deg, halos_data.DEC*u.deg, frame='icrs')

for i, cur_BCG in enumerate(BCG_data):
    cur_ID = cur_BCG['HALOID']
    cur_RA = cur_BCG['TRA']
    cur_DEC = cur_BCG['TDEC']
    ind, ang_sep = find_nearest(i)
    z_sep = np.abs(cur_BCG['Z'] - halos_data.Z[ind])

    #ind = np.argmin(np.abs(cur_RA-halos_RA)) #this speeds up the calculation, asssuming minRA gives min_distance
    
    halo_ID = halos_data.ID[ind]
    RA_val = halos_data.RA[ind]
    DEC_val = halos_data.DEC[ind]
    #ang_sep = np.sqrt((RA_val-cur_RA)**2 + (DEC_val-cur_DEC)**2)

    if (ang_sep <= ang_lim) & (z_sep <= z_lim): 
        BCG_matched_ID.append(cur_ID)
        halo_matched_ID.append(halo_ID)
        
    if i % 10 == 0: 
        print "Processing BCG number ", i
        print "Number of centered are", len(BCG_matched_ID)
print "Total number of centered clusters are", len(BCG_matched_ID)
print "Out of total number of clusters of ", len(BCG_data)

Processing BCG number  0
Number of centered are 1
Processing BCG number  10
Number of centered are 11
Processing BCG number  20
Number of centered are 21
Processing BCG number  30
Number of centered are 31
Processing BCG number  40
Number of centered are 41


KeyboardInterrupt: 

## Add matched BCG Lensed RA, lensed DEC onto halos_data record array

## Writing to record array

## Randomly sample halos to produce halo shape

## Writing to record array


## Testing

In [ ]:
#halos_cut = np.where((halos_rand['Z']>0.32))
#halos_rand = halos_rand[halos_cut]

r = np.sqrt(halos_rand.PX**2 + halos_rand.PY**2 + halos_rand.PZ**2)
rbin = r//25
print np.max(rbin), np.max(halos_rand.Z)
rbin_ind = np.where(rbin == 72)
halos_cut = halos_rand[rbin_ind]
print halos_cut.Z
print np.max(halos_cut.Z), np.min(halos_cut.Z)
#print rbin, np.max(rbin), np.min(rbin)
#print np.max(halos_rand.Z), np.min(halos_rand.Z)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(halos_cut.TRA, halos_cut.TDEC)
plt.show()

In [ ]:
## Testing for coordinate matching between PX, PY, PZ and RA, DEC, Z
from astropy.coordinates import Distance
from astropy import units as u
from astropy import cosmology
from astropy.cosmology import WMAP7 as cosmo
from astropy.coordinates import SkyCoord

#Select sample halo.
testn= 1000
test_halo = halos_cut[testn]

d_halo = cosmo.comoving_distance([test_halo.Z])*0.702  
c = SkyCoord(ra=test_halo.TRA*u.degree, dec=test_halo.TDEC*u.degree, distance=d_halo*u.Mpc)[0]
  
ptcl_c = SkyCoord(x=-1073.3064, y=483.5157, z=451.49063, representation_type='cartesian')
print ptcl_c

#print c, d_halo
#print c.cartesian.x.value, c.cartesian.y, c.cartesian.z
#print test_halo.PX, test_halo.PY, test_halo.PZ
#print np.sqrt((c.cartesian.x.value-test_halo.PX)**2. + (c.cartesian.y.value-test_halo.PY)**2. \
#              + (c.cartesian.z.value-test_halo.PZ)**2.)


#Set to transverse or line of sight comoving distance? Can't tell. 
#comoving_r = cosmo.comovingDistance(zmin, zmax, transverse=True)